In [42]:
import pandas as pd
import tensorflow as tf

def preprocess_data(data, name=["name"]):
    data_processed = data.drop("Unnamed: 0", axis=1)
    # data_processed = data.drop(name[0], axis=1)
    data_processed = data_processed.fillna(0)
    # Text to binary
    data_processed = pd.get_dummies(data_processed, columns=name)

    return data_processed.values

def main():
    # Get data
    ovens_data = pd.read_csv("ovens_done.csv")
    series_data = pd.read_csv("series_done.csv")

    # Preprocess
    processed_ovens_data = preprocess_data(ovens_data)
    processed_series_data = preprocess_data(series_data, ["№слитка-индекс-часть"])

    # Make valid and test data
    split_index = len(processed_series_data) // 2

    validation_ovens_data = processed_ovens_data[:split_index]
    validation_series_data = processed_series_data[:split_index]

    test_ovens_data = processed_ovens_data[split_index:]
    test_series_data = processed_series_data[split_index:]

    batch_size = 32
    sequence_length =

    # Data generators
    train_data_generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(processed_ovens_data, processed_series_data, length=sequence_length, batch_size=batch_size)
    validation_data_generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(validation_ovens_data, validation_series_data, length=sequence_length, batch_size=batch_size)
    test_data_generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(test_ovens_data, test_series_data, length=sequence_length, batch_size=batch_size)

    # Neural network
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, return_sequences=True, input_shape=(sequence_length, processed_ovens_data.shape[1])),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(processed_series_data.shape[1], activation='softmax'))
    ])

    # Model compile
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Model fit
    model.fit(train_data_generator, epochs=10, validation_data=validation_data_generator)

    # Result
    loss, accuracy = model.evaluate(test_data_generator)
    print(f"Точность модели на тестовой выборке: {accuracy}")


if __name__ == "__main__":
    main()


ValueError: ignored